In [1]:
import pandas as pd
import numpy as np
import config
import datetime
from sqlalchemy import text
import plotly
import plotly.express as px
import ipywidgets
from urllib.request import urlopen
import json
import plotly.graph_objects as go
import geopandas as gpd
import warnings
import pyproj
import plotly.io as pio
print(config.version)
print(config.connection_string)
print(plotly.__version__, ipywidgets.__version__, pd.__version__, np.__version__)
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore', category=UserWarning) 

3.2.0.Dev
postgresql://postgres:sa@localhost/icip
6.3.1 8.1.7 2.0.0 1.26.4


In [2]:
# Charger et filtrer les données
gdf = gpd.read_file("contours-iris.gpkg")
gdf = gdf[gdf["nom_commune"] == "Nice"].to_crs(epsg=4326)
gdf_l93 = gdf.to_crs(2154)
pts_l93 = gdf_l93.geometry.representative_point()
pts = pts_l93.to_crs(4326)
gdf["fid"] = gdf.index
gdf["lon"] = gdf.geometry.centroid.x
gdf["lat"] = gdf.geometry.centroid.y
gdf=gdf.sort_values(by="code_iris")
geojson=gdf.__geo_interface__
gdf

cleabs code_insee nom_commune  iris  code_iris   
2597   IRIS____0000000060880101      06088        Nice  0101  060880101  \
4458   IRIS____0000000060880102      06088        Nice  0102  060880102   
5715   IRIS____0000000060880103      06088        Nice  0103  060880103   
9348   IRIS____0000000060880201      06088        Nice  0201  060880201   
9785   IRIS____0000000060880202      06088        Nice  0202  060880202   
...                         ...        ...         ...   ...        ...   
14763  IRIS____0000000060883501      06088        Nice  3501  060883501   
14773  IRIS____0000000060883601      06088        Nice  3601  060883601   
14774  IRIS____0000000060883602      06088        Nice  3602  060883602   
14786  IRIS____0000000060883701      06088        Nice  3701  060883701   
14798  IRIS____0000000060883801      06088        Nice  3801  060883801   

                         nom_iris type_iris   
2597               Saleya-Château         H  \
4458   Vieux Nice-Sainte-Réparate         H   
5715        Vieux Nice-Visitation         H   
9348                    Garibaldi         H   
9785                         Port         H   
...                           ...       ...   
14763               Crémat-Bellet         H   
14773       Saint-Pierre de Féric         H   
14774                    Costière         H   
14786    Saint-Pancrace-Pessicart         H   
14798                     Gairaut         H   

                                                geometry    fid       lon   
2597   MULTIPOLYGON (((7.28193 43.69312, 7.28159 43.6...   2597  7.277064  \
4458   MULTIPOLYGON (((7.27454 43.69773, 7.27598 43.6...   4458  7.275641   
5715   MULTIPOLYGON (((7.27842 43.70006, 7.27884 43.7...   5715  7.279384   
9348   MULTIPOLYGON (((7.27904 43.70126, 7.28007 43.7...   9348  7.281249   
9785   MULTIPOLYGON (((7.29038 43.69227, 7.29029 43.6...   9785  7.285948   
...                                                  ...    ...       ...   
14763  MULTIPOLYGON (((7.20972 43.75747, 7.21002 43.7...  14763  7.210291   
14773  MULTIPOLYGON (((7.22218 43.72576, 7.22231 43.7...  14773  7.232767   
14774  MULTIPOLYGON (((7.22543 43.71696, 7.22593 43.7...  14774  7.235730   
14786  MULTIPOLYGON (((7.222 43.74145, 7.22215 43.741...  14786  7.237896   
14798  MULTIPOLYGON (((7.25138 43.73985, 7.25192 43.7...  14798  7.264822   

             lat  
2597   43.695476  
4458   43.697293  
5715   43.698417  
9348   43.702325  
9785   43.695477  
...          ...  
14763  43.733731  
14773  43.720739  
14774  43.709542  
14786  43.735819  
14798  43.737591  

[146 rows x 11 columns]

In [3]:
year=24
sql = f"""
    select e.nofinesset, e.rs, an.id an_id, an.score, an.lon, an.lat, an.iris, i.id, c.code code_commune, c.nom from etablissement e
join etablissement_date_source eds on eds.etablissement_id=e.id
join adresse_raw ar on ar.id=e.adresse_raw_id
join adresse_norm an on an.id=ar.adresse_norm_id
join iris.iris i on i.code=an.iris
join iris.commune c on c.id=i.commune_id
where categetab=620
and date_source_id={year}12
and c.code='06088'
"""
etab=pd.read_sql(sql, config.connection_string)
etab["pos"]=etab["an_id"].astype(str)+"_"+etab["rs"]+"_"+etab["score"].astype(str)+"_"+etab["lon"].astype(str)+"_"+etab["lat"].astype(str)
etab

nofinesset                                   rs   an_id   score       lon   
0    060014560            SELARL PHARMACIE FIANDINO  101640  0.7000  7.265588  \
1    060014594             SELARL PHARMACIE VERDIER  105222  1.0000  7.256275   
2    060014610                    PHARMACIE SAETONE  105223  1.0000  7.264493   
3    060014644               PHARMACIE TOMASI-DAVID  105225  1.0000  7.251111   
4    060014677                PHARMACIE DU VAL D'OR  105226  0.9495  7.301278   
..         ...                                  ...     ...     ...       ...   
144  060019031                       PHARMACIE TATU  105408  1.0000  7.265028   
145  060019080               SARL PHARMACIE BOUJMIL  105410  1.0000  7.261588   
146  060019130  SELARL PHIE CHARPENEL - VICTOR HUGO  105412  1.0000  7.259882   
147  060019163         PHARMACIE DE LA VOIE ROMAINE  105413  1.0000  7.265587   
148  060019197   SELARL L'APOTHEQK DU SQUARE GOIRAN  105415  0.9495  7.249910   

           lat       iris        id code_commune   nom   
0    43.696041  060882201  60882201        06088  Nice  \
1    43.700493  060882302  60882302        06088  Nice   
2    43.704099  060882202  60882202        06088  Nice   
3    43.706480  060881903  60881903        06088  Nice   
4    43.735194  060880903  60880903        06088  Nice   
..         ...        ...       ...          ...   ...   
144  43.703303  060882202  60882202        06088  Nice   
145  43.706300  060881803  60881803        06088  Nice   
146  43.698282  060882402  60882402        06088  Nice   
147  43.696041  060882201  60882201        06088  Nice   
148  43.725430  060881305  60881305        06088  Nice   

                                                   pos  
0    101640_SELARL PHARMACIE FIANDINO_0.7_7.265588_...  
1    105222_SELARL PHARMACIE VERDIER_1.0_7.256275_4...  
2    105223_PHARMACIE SAETONE_1.0_7.264492565958998...  
3    105225_PHARMACIE TOMASI-DAVID_1.0_7.2511112885...  
4    105226_PHARMACIE DU VAL D'OR_0.949500000000000...  
..                                                 ...  
144       105408_PHARMACIE TATU_1.0_7.265028_43.703303  
145  105410_SARL PHARMACIE BOUJMIL_1.0_7.2615883306...  
146  105412_SELARL PHIE CHARPENEL - VICTOR HUGO_1.0...  
147  105413_PHARMACIE DE LA VOIE ROMAINE_1.0_7.2655...  
148  105415_SELARL L'APOTHEQK DU SQUARE GOIRAN_0.94...  

[149 rows x 11 columns]

In [4]:
sql=f"""
with ir as (
	select i.code, count(distinct (an.lon, an.lat)) --count(e.id)
	from etablissement e
	join etablissement_date_source eds on eds.etablissement_id=e.id
	join adresse_raw ar on ar.id=e.adresse_raw_id
	join adresse_norm an on an.id=ar.adresse_norm_id
	join iris.iris i on i.code=an.iris
	join iris.commune c on c.id=i.commune_id
	where categetab=620
	and eds.date_source_id={year}12
	and c.code='06088'
	group by i.id
)
select i.code code_iris, coalesce(ir.count,0) nb, pi.pop, coalesce(ir.count,0)*100000/pi.pop::float ratio from iris.iris i
join iris.commune c on c.id=i.commune_id
left outer join ir on ir.code=i.code
join iris.pop_iris pi on pi.iris_id=i.id and pi.year=21
where c.code='06088'
order by code_iris
"""
pop=pd.read_sql(sql, config.connection_string)
pop

code_iris  nb          pop       ratio
0    060880101   2  1348.551994  148.307222
1    060880102   3  1302.167160  230.385168
2    060880103   1  1536.126830   65.098791
3    060880201   3  2260.231562  132.729763
4    060880202   1  1857.073057   53.848178
..         ...  ..          ...         ...
141  060883501   0  2649.665320    0.000000
142  060883601   0  2930.552478    0.000000
143  060883602   0  2200.226082    0.000000
144  060883701   0  3188.198432    0.000000
145  060883801   1  2685.623135   37.235306

[146 rows x 4 columns]

In [5]:
gdf = gdf.merge(pop, on="code_iris", how="left", suffixes=('', '')).copy()
gdf

cleabs code_insee nom_commune  iris  code_iris   
0    IRIS____0000000060880101      06088        Nice  0101  060880101  \
1    IRIS____0000000060880102      06088        Nice  0102  060880102   
2    IRIS____0000000060880103      06088        Nice  0103  060880103   
3    IRIS____0000000060880201      06088        Nice  0201  060880201   
4    IRIS____0000000060880202      06088        Nice  0202  060880202   
..                        ...        ...         ...   ...        ...   
141  IRIS____0000000060883501      06088        Nice  3501  060883501   
142  IRIS____0000000060883601      06088        Nice  3601  060883601   
143  IRIS____0000000060883602      06088        Nice  3602  060883602   
144  IRIS____0000000060883701      06088        Nice  3701  060883701   
145  IRIS____0000000060883801      06088        Nice  3801  060883801   

                       nom_iris type_iris   
0                Saleya-Château         H  \
1    Vieux Nice-Sainte-Réparate         H   
2         Vieux Nice-Visitation         H   
3                     Garibaldi         H   
4                          Port         H   
..                          ...       ...   
141               Crémat-Bellet         H   
142       Saint-Pierre de Féric         H   
143                    Costière         H   
144    Saint-Pancrace-Pessicart         H   
145                     Gairaut         H   

                                              geometry    fid       lon   
0    MULTIPOLYGON (((7.28193 43.69312, 7.28159 43.6...   2597  7.277064  \
1    MULTIPOLYGON (((7.27454 43.69773, 7.27598 43.6...   4458  7.275641   
2    MULTIPOLYGON (((7.27842 43.70006, 7.27884 43.7...   5715  7.279384   
3    MULTIPOLYGON (((7.27904 43.70126, 7.28007 43.7...   9348  7.281249   
4    MULTIPOLYGON (((7.29038 43.69227, 7.29029 43.6...   9785  7.285948   
..                                                 ...    ...       ...   
141  MULTIPOLYGON (((7.20972 43.75747, 7.21002 43.7...  14763  7.210291   
142  MULTIPOLYGON (((7.22218 43.72576, 7.22231 43.7...  14773  7.232767   
143  MULTIPOLYGON (((7.22543 43.71696, 7.22593 43.7...  14774  7.235730   
144  MULTIPOLYGON (((7.222 43.74145, 7.22215 43.741...  14786  7.237896   
145  MULTIPOLYGON (((7.25138 43.73985, 7.25192 43.7...  14798  7.264822   

           lat  nb          pop       ratio  
0    43.695476   2  1348.551994  148.307222  
1    43.697293   3  1302.167160  230.385168  
2    43.698417   1  1536.126830   65.098791  
3    43.702325   3  2260.231562  132.729763  
4    43.695477   1  1857.073057   53.848178  
..         ...  ..          ...         ...  
141  43.733731   0  2649.665320    0.000000  
142  43.720739   0  2930.552478    0.000000  
143  43.709542   0  2200.226082    0.000000  
144  43.735819   0  3188.198432    0.000000  
145  43.737591   1  2685.623135   37.235306  

[146 rows x 14 columns]

In [6]:
# Créer la figure

hover_enabled=True

fig = go.Figure()

z_values=np.log10(gdf["ratio"]+0.1)

fig.add_trace(go.Choropleth(
    geojson=geojson,
    locations=gdf["fid"],
    z=z_values,
    zmin=z_values.min(),
    zmax=z_values.max(),
    text=gdf.apply(
        lambda row: f"Iris: {row['code_iris']}<br>Nom: {row['nom_iris']}<br>Nb: {row['nb']}<br>Population: {np.round(row['pop'],0):.0f}<br>Densité: {np.round(row['ratio'],0):.0f}/100k",
        axis=1
    ),
    hoverinfo="text",
    showscale=True,
    # colorscale = [
    #     [0.0, "#2166ac"],   # bleu foncé
    #     [0.0001, "#67a9cf"],   # bleu moyen
    #     [0.001, "#d1e5f0"],   # bleu très clair
    #     [0.01, "#f7f7f7"],   # blanc
    #     [0.1, "#fddbc7"],   # rose très clair
    #     [0.15, "#ef8a62"],   # rouge moyen
    #     [0.2, "#b2182b"],   # rouge foncé
    #     [1.0, "#ff0000"]    # rouge foncé
    # ],
    colorscale="RdBu_r",
    colorbar={"title": "Density: 10eNb/100k"}
))

center_lat = gdf.geometry.centroid.y.mean()
center_lon = gdf.geometry.centroid.x.mean()

# Ajouter les étiquettes
gdf_l93 = gdf.to_crs(2154)
pts_l93 = gdf_l93.geometry.representative_point()
pts = pts_l93.to_crs(4326)

fig.add_trace(go.Scattergeo(
    lon=etab["lon"],
    lat=etab["lat"],
    text=etab["pos"],
    hoverinfo="text" if hover_enabled else "skip",
    showlegend=False,
    marker=dict(
            color='#99ff99',
            size=8,
            line=dict(
                color='#006600',
                width=2
            ),
        ),
))

# Mise en page
fig.update_layout(
    geo=dict(
        projection_type="mercator",
        center={"lat": center_lat, "lon": center_lon},
        fitbounds="locations",
        showland=False,
        showcountries=False,
        showocean=False,
        showlakes=False,
        showrivers=False,
        visible=False,
    ),
    title=f"Répartition des Pharmacies par IRIS- 20{year} - Ville de Nice",
    height=1000,
    width=1200,
    margin=dict(l=0, r=0, t=50, b=0),
)

fig.show()

In [7]:
pio.write_html(fig, file=f"nice_pharma_20{year}.html", auto_open=True, include_plotlyjs="cdn", full_html=True, config={"responsive": True})